In [1]:
from flash.image import SemanticSegmentation
from flash.image import SemanticSegmentationData


In [2]:
%matplotlib inline
import cv2
import numpy as np
import torch
import cv2
from matplotlib import pyplot as plt
from flash import Trainer


In [3]:

model = SemanticSegmentation.load_from_checkpoint(r"E:\Google Drive\Acads\Notes\final sem\ML\project\learning-to-parse-pdf\documentAnalysis\flash\semantic_segmentation_model.pt")


Using 'efficientnet-b0' provided by qubvel/segmentation_models.pytorch (https://github.com/qubvel/segmentation_models.pytorch).
Using 'unet' provided by qubvel/segmentation_models.pytorch (https://github.com/qubvel/segmentation_models.pytorch).


In [32]:
dm = SemanticSegmentationData.from_folders(
    predict_folder=r"E:\Google Drive\Acads\Notes\final sem\ML\project\learning-to-parse-pdf\documentAnalysis\data\images/outofdata/",
    transform_kwargs=dict(image_size=(800, 640)),
    batch_size = 1,
    )
trainer = Trainer(max_epochs=1, gpus = 1)
predictions = trainer.predict(model, dm)



GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Predicting: 100%|██████████| 7/7 [00:02<00:00,  3.11it/s]


In [33]:
print('Total images', len(predictions))
for i in range(len(predictions)):
    print(predictions[i][0]['metadata'])
    inp = (predictions[i][0]['input']).numpy().copy()
    inp = np.transpose(inp, (1, 2, 0))
    inp = cv2.resize(inp, (640, 800))
    pred = (predictions[i][0]['preds'])
    pred = torch.softmax(pred, dim = 0)
    pred = torch.argmax(pred, dim = 0).numpy().astype(np.uint8)
    pred = cv2.resize(pred, (640, 800))
    inp = (inp/np.max(inp) * 255).astype(np.uint8)
    inp[:, :, 0][pred == 1] = 128
    inp[:, :, 1][pred == 2] = 128
    inp[:, :, 2][pred == 3] = 128
    inp[:, :, 1][pred == 4] = 55
    inp[:, :, 2][pred == 5] = 55
    inp[:, :, 0][pred == 6] = 55
    inp[:, :, 1][pred == 7] = 200

    b = inp
    c =  (pred/np.max(pred) * 255).astype(np.uint8)
    c = np.stack((c, c, c), axis = 2)
    out = np.hstack(( b, c))
    cv2.imwrite(f'preds/out_{i}.jpg', b)


Total images 7
{'size': [1099, 720], 'filepath': 'E:\\Google Drive\\Acads\\Notes\\final sem\\ML\\project\\learning-to-parse-pdf\\documentAnalysis\\data\\images/outofdata/E4T1Za3VcAEPtMt.jpg'}
{'size': [1650, 1275], 'filepath': 'E:\\Google Drive\\Acads\\Notes\\final sem\\ML\\project\\learning-to-parse-pdf\\documentAnalysis\\data\\images/outofdata/Indian Classical Music Synthesis_page-0002.jpg'}
{'size': [1650, 1275], 'filepath': 'E:\\Google Drive\\Acads\\Notes\\final sem\\ML\\project\\learning-to-parse-pdf\\documentAnalysis\\data\\images/outofdata/Memory_Guided_Road_Segmentation___ICIAP_page-0005.jpg'}
{'size': [380, 261], 'filepath': 'E:\\Google Drive\\Acads\\Notes\\final sem\\ML\\project\\learning-to-parse-pdf\\documentAnalysis\\data\\images/outofdata/NewsPaperTheHindu.jpg'}
{'size': [1497, 1058], 'filepath': 'E:\\Google Drive\\Acads\\Notes\\final sem\\ML\\project\\learning-to-parse-pdf\\documentAnalysis\\data\\images/outofdata/TIMES-OF-INDIA-16-MAY-pdf (1).jpg'}
{'size': [853, 657], 